In [26]:
import sys
sys.path.append("..")
import lib
import numpy as np

## Outline Problem
- CPU has single register X starting with value 1
- Two instructions
    - addx V
        - takes 2 cycles to complete
        - after two cycles, X register adjusts by value V
    - noop
        - takes 1 cycle to complete

For now, try to look at signal strength ($n_{cycle} \times val_{register}$) at each 20th cycle.

## Task of problem A:
- Sum up the signal strengths during 20th, 60th, 100th cycle and so on ...

## Addition of part B:
- Register X controls horizontal position of sprite -> specifically the middle of the 3-pixels-wide sprite
- pixels on CRT: 40 wide and 6 high -> CRT draws top row left-to-right, then row below that and so on (240 cycle)
- left most is in position 0 and right-most pixel in position 39
- CRT draws single pixel during each cycle

- if sprite positioned s.t. 1/3 of its pixels is the one currently being drawn -> the screen produces a LIT pixel (#)
- else a NON-LIT pixel (.)

## Task of problem B:
Just read the screen.

In [46]:
class ClockCircuit():
    def __init__(self, highlighter : str = '#', lowlighter : str = '.') -> None:
        self.cycle = 0
        self.value = 1 # middle of 3-width sprite -> starts at 012
        self.tracker_partA = 0 # sums up each 20th, 60th, 100th, ... signal strength
        self.CRT_Screen = '' # output of CRT
        self.CRT_Writer = 0 # position of CRT writer
        self.HIGHLIGHTER = highlighter #custom #
        self.LOWLIGHTER = lowlighter #custom .

    def __repr__(self) -> None:
        return f"Cycle {self.cycle} and Value {self.value}"

    def update_partA(self) -> None:
        if self.cycle % 40 == 20:
            self.tracker_partA += self.cycle * self.value
        return

    def print_CRT(self, verbose : bool = False) -> None:
        print(f"Printing CRT with length {len(self.CRT_Screen)} at cycle {self.cycle}") if verbose else None
        LINE_LENGTH = 40
        for idx_line in range(len(self.CRT_Screen) // LINE_LENGTH + 1):
            print(self.CRT_Screen[idx_line * 40 : (idx_line+1) * 40])
        return
    
    def draw_on_CRT(self, verbose : bool = False) -> None:
        # compare positions of sprite (1 around register self.value) with CRT_writer
        # draw # if hovering, else .
        if np.absolute(self.CRT_Writer % 40 - self.value) <= 1:
            self.CRT_Screen += self.HIGHLIGHTER
        else:
            self.CRT_Screen += self.LOWLIGHTER
        self.print_CRT(True) if verbose else None
        self.CRT_Writer += 1 # increase at end of cycle
        return

    def execute_move(self, move : str, change_of_value : int = None, verbose : bool = False) -> None:
        # executes move noop or addx, if addx, includes change of value
        self.cycle += 1
        print(f"Executing {move} with {change_of_value} value delta at cycle {self.cycle}.") if verbose else None
        # implicit: begin executing addx
        self.draw_on_CRT(verbose)
        self.update_partA()
        if move == "addx":
            self.cycle += 1
            self.draw_on_CRT(verbose)
            self.update_partA()
            self.value += change_of_value # explicit: finishing executing addx
        return

    def parse_line(self, line : str, verbose : bool = False) -> None:
        # parses line and passes to execute_move
        change_of_value = 0 if line == "noop" else int(line[5:])
        self.execute_move("noop", verbose) if line == "noop" else self.execute_move("addx", change_of_value, verbose)
        return

In [47]:
def compute_partA(filename : str, verbose : bool = False) -> int:
    lines = lib.read_file(filename)
    instance = ClockCircuit()
    for line in lines:
        instance.parse_line(line, verbose)
    return instance.tracker_partA

def solve_partA():
    result = compute_partA("test_input.txt", False)
    print(f"Result of part A on test file : {result}")
    assert result == 13140, f"Part A faulty on test file... output = {result}"
    print("Part A works for test file, moving on to whole input...")
    result = compute_partA("input.txt", False)
    print(f"Answer: {result}")
    return

solve_partA()

Result of part A on test file : 13140
Part A works for test file, moving on to whole input...
Answer: 11780


In [69]:
def compute_partB(filename : str, verbose : bool = False) -> int:
    lines = lib.read_file(filename)
    instance = ClockCircuit('#', ' ')
    for line in lines:
        instance.parse_line(line, verbose)
    return instance

def solve_partB():
    result = compute_partB("test_input.txt", False)
    print(f"Result of part B on test file :")
    result.print_CRT()
    print("Part B works for test file, moving on to whole input...")
    result = compute_partB("input.txt", False)
    print(f"Answer :")
    result.print_CRT(True)
    return

solve_partB()

Result of part B on test file :
##  ##  ##  ##  ##  ##  ##  ##  ##  ##  
###   ###   ###   ###   ###   ###   ### 
####    ####    ####    ####    ####    
#####     #####     #####     #####     
######      ######      ######      ####
#######       #######       #######     

Part B works for test file, moving on to whole input...
Answer :
Printing CRT with length 240 at cycle 240
###  #### #  # #    ###   ##  #  #  ##  
#  #    # #  # #    #  # #  # #  # #  # 
#  #   #  #  # #    ###  #  # #  # #  # 
###   #   #  # #    #  # #### #  # #### 
#    #    #  # #    #  # #  # #  # #  # 
#    ####  ##  #### ###  #  #  ##  #  # 

